# V4
Great! It seems json make everything better in previous example.
Now we complete it with an additional translation requirement.

**Note**: if you get gemini error `finish_reason: RECITATION`  
This might be a copyright warning.  
Just retry with another llm, say: openai

## Gemini Pro API and OpenAI API
Check [Bilingual-NovelTranslator-v1.ipynb](Bilingual-NovelTranslator-v1.ipynb) for more information.

**Note** : Remember to fill you api key in `.env` file

In [1]:
%load_ext autoreload
%autoreload 2
    
import utils

geminipro, openai = utils.init_llm()

In [2]:
def split_paragraph(source_path):
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,
        chunk_overlap=0,
        separators= ["\n\n\n\n","\n\n\n","\n\n"]
        # length_function=len,
        # is_separator_regex=True,
    )
    content=""
    with open(source_path, 'r') as f:
        content = f.read()
    
        
    return text_splitter.split_text(content)

In [3]:
def autoprocess_paragraph(llm, paragraphs, target_file=None, start=0):
    from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
    from langchain_core.output_parsers.string import StrOutputParser
    prompt = PromptTemplate.from_template(
        """You are a professional translation assistant. Your task is to translate a paragraph from english to traditional Chinese.

There will be an OCR document with many layout issues.
You should first try to find out every complete sentence in the article,
and check and fix it if the sentence got split(linebreak) by mistake.
Your shoud check and fix misspellings and typos a well.

Then you should translate each sentence into zh-tw.
If a sentence is nonsense, make the translation identical to original source.

The output should be a markdown code snippet formatted in the following json schema, including the leading and trailing "```json" and "```":

```json
[
    {{
        "correction": string  // the sentence after error correction
        "zh-tw": string  // the translated sentence zh-tw
    }},
    {{
        "correction": string  // the sentence after error correction
        "zh-tw": string  // the translated sentence zh-tw
    }},
    .......
    ........
    {{
        "correction": string  // the sentence after error correction
        "zh-tw": string  // the translated sentence zh-tw
    }}
]
```

Example:
---
input:

Eefore one girl and another even younger one stood a figure in 
full p1ate armor brandishing a 5word.Lhe blade swung, sparkl-
ing in the sun1ight as if to say that taking their lives in a 
single stroke would be an act of mercy.

---
output:

```json
[
    {{
        "correction": "Before one girl and another even younger one stood a figure in full plate armor brandishing a sword.",
        "zh-tw": "在一名少女以及比她更年輕的少女面前，站著一位身穿全身板甲、揮舞著劍的男子。"
    }},
    {{
        "correction": "The blade swung, sparkling in the sunlight as if to say that taking their lives in a single stroke would be an act of mercy."
        "zh-tw": "刀鋒揮動，在陽光下閃爍，彷彿在說一刀奪命是仁慈的作為。"
    }}
]
```
---


The original paragraph is as following:

{input}

""")
    print(f"using llm: {llm}")
    chain = prompt | llm | StrOutputParser()
    translated=[]
    from tqdm.notebook import tqdm, trange
    errorindx = -1
    with tqdm(total=len(paragraphs)) as progress_bar:
        for i, paragraph in enumerate(paragraphs):
            if i < start:
                progress_bar.update(1)
                continue
            if len(paragraph.strip()) > 0:
                try:
                    # temp = ""
                    # for chunk in chain.stream({"input": paragraph}):
                    #     print(chunk, end="", flush=True)
                    #     temp += chunk
                    # print("\n------\n")    
                    temp = chain.invoke({"input": paragraph})
                except Exception as e:
                    print(f"Error occurs at {i}:")
                    print(paragraph)
                    print(e)
                    errorindx = i
                    return errorindx,translated;
                translated.append(temp)
                print(temp)
                print("\n------\n")    
                if target_file:
                    with open(target_file, 'a+') as f:
                        f.write(temp)
                        f.write('\n')
            progress_bar.update(1)
    return errorindx,translated;

In [4]:
source_filenames = ["alice_in_wonderland.txt"]

In [5]:
errorbookindex=-1
errorindx=-1

In [6]:
for i, source_file in enumerate(source_filenames):
    if i < errorbookindex:
        continue
    print(source_file)
    path = source_file
    paragraphs = split_paragraph(path)[1:5]

    progress=0
    while (progress < len(paragraphs)):
        errorindx, translated_paragraphs = autoprocess_paragraph(geminipro, paragraphs, start=errorindx)        
        if errorindx < 0:
            progress=len(paragraphs)
        else: # errorindx >= 0:# error, retry
            _errorindx = -1
            for _llm in [openai]: # use optional llm to retry
                _paragraphs = paragraphs[errorindx:errorindx+1]
                _errorindx, _translated_paragraphs = autoprocess_paragraph(_llm, _paragraphs, start=0)
                if _errorindx < 0: # retry success
                    break
            if _errorindx < 0: # any retry success
                errorindx=errorindx+1 # continue next
                progress = errorindx
                continue

    if errorindx >= 0:
        errorbookindex=i
        break
        

alice_in_wonderland.txt
using llm: model='gemini-pro' temperature=0.5 top_p=0.85 safety_settings={<HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>} client= genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
) convert_system_message_to_human=True


  0%|          | 0/4 [00:00<?, ?it/s]

```json
[
    {
        "correction": "CHAPTER I",
        "zh-tw": "第一章"
    },
    {
        "correction": "Down the Rabbit-Hole",
        "zh-tw": "墜入兔子洞"
    }
]
```

------

```json
[
  {
    "correction": "Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do.",
    "zh-tw": "愛麗絲開始覺得坐在河岸邊，陪伴姊姊，卻無所事事感到非常無聊。"
  },
  {
    "correction": "Once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it.",
    "zh-tw": "她曾經一兩次偷瞄姊姊正在讀的書，但書中沒有插圖或對話。"
  },
  {
    "correction": "And what is the use of a book, thought Alice, without pictures or conversation?",
    "zh-tw": "愛麗絲心想，一本沒有插圖或對話的書有什麼用呢？"
  },
  {
    "correction": "So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies.",
    "zh-tw": "因此她開始思考（儘管她因為天氣炎熱而感到昏昏欲睡和遲鈍